[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BouleJaune/picselliaT/blob/master/notebookOBDTC.ipynb)



# Object-detection made easy
We will learn how to easily train an object detection model from a list of pre-trained models with the dataset you created on the picsell-IA platform.

In [ ]:
!git clone https://github.com/BouleJaune/picselliaT
%cd picselliaT/
!pip install picsellia
%tensorflow_version 1.x

In [ ]:
import gdown
gdown.download("https://drive.google.com/u/0/uc?export=download&confirm=sWbX&id=14UDmfWrnP2jwppxvcL6wkjPtObuuMh20", "models.zip", quiet=False)
!unzip models.zip

In [ ]:
import tensorflow as tf
tf.__version__

## Imports

In [ ]:
import sys
sys.path.append("slim")
from picsellia import Client
import main
from util.infer import infer
import tensorflow as tf





## List of possible models : 

- Mask RCNN, a segmentation model. String variable: 'mask_rcnn'
- Faster RCNN an accurate but slow object detection model. String variable: 'faster_rcnn'
- SSD Inception, a fast but less accurate object detection model. String variable : 'ssd_inception'

## Setup

We need to start setting up some variables before anything else.

In [ ]:
project_token = "d8e83b34-352e-41b1-9329-03a791f8d4ef" # Token from the picsell-IA platform
api_token = "5890ad3e6a9701cd123ef91b69ad396de8c2d20a"
model_name = "test_model" # Name your to-be trained model
annotation_type = "polygon" # Chose the type of annotation used

batch_size = 1
learning_rate = None #You can let this value to None
nb_steps = 10

## Client initialisation and data pre-processing

We communicate with the platform to create a new model and get the images and annotations.
With this we can generate the label map, smartly split our data then create the TFRecord files which will be used as input for the model. 

In [ ]:
clt = Client(api_token=api_token, host="https://backstage.picsellia.com/sdk/")
clt.init_project(project_token=project_token)
clt.init_model(model_name)

clt.dl_annotations()
clt.generate_labelmap()
clt.local_pic_save()

In [ ]:
main.create_record_files(label_path=clt.label_path, record_dir=clt.record_dir, 
                         tfExample_generator=clt.tf_vars_generator, annotation_type=annotation_type)

Here we edit the base protobuf configuration of our model with our parameters. 
We check if it the first training on this model to see if we should train from a previous checkpoint or not.

In [ ]:
main.edit_config(model_selected=model_selected, config_output_dir=clt.config_dir,
            record_dir=clt.record_dir, 
            label_map_path=clt.label_path, 
            masks=mask_type, 
            num_steps=nb_steps,
            batch_size=batch_size, 
            learning_rate=learning_rate,
            training_id=clt.training_id)

## Training

We can launch the training... and it's as easy as just telling the fonction where is the configuration file and where we want the checkpoints and records to be saved !

In [ ]:
main.train(ckpt_dir=clt.checkpoint_dir, 
                     conf_dir=clt.config_dir)

Now that the training has ended we want to send the logs to our dashboard so we can neatly see the sweet decrease of the loss. 

In [ ]:
dict_log = main.tfevents_to_dict(path=clt.checkpoint_dir)
clt.send_logs(dict_log)

## Exporting and infering

The model is trained but we still need to export it to a Tensorflow graph proto to use it.

In [ ]:
tf.reset_default_graph()
main.export_infer_graph(ckpt_dir=clt.checkpoint_dir, 
                       exported_model_dir=clt.exported_model, 
                       pipeline_config_path=clt.config_dir,
                       write_inference_graph=True, input_type="image_tensor", input_shape=None)

Now we will use the exported model to do some inference on our evaluation set, then send the results to the dashboard.
You can set the minimum confidence treshold at which we keep the bounding boxes to the value you like the most.

In [ ]:
min_score_thresh = 0.3 
infer(clt.eval_list, exported_model_dir=clt.exported_model, 
          label_map_path=clt.label_path, results_dir=clt.results_dir, min_score_thresh=min_score_thresh, num_infer=16)
clt.send_examples()

In [ ]:
clt.send_weights(clt.exported_model+'saved_model/saved_model.pb')